In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Análisis de la de los resultados de la minería de reglas

In [2]:
import re
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, size, split
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import size

# Preparar dataset con diagnósticos como consecuente para análisis

In [3]:
# Crear sesión de Spark
spark = SparkSession.builder.appName("ReglasDiagnostico").getOrCreate()

In [4]:
ruta_rules = "drive/MyDrive/TFM_febrero2025/fpgrowth_rules_parquet"

df_spark = spark.read.parquet(ruta_rules, header=True, inferSchema=True)

In [5]:
# Filtrar reglas con un diagnóstico como consecuente: Letra + 2 números

# Expresión regular para identificar diagnósticos
diagnosis_pattern = r"^[A-Za-z][0-9]{2}$"

@udf(BooleanType())
def has_diagnosis_udf(consequent_items):
    if isinstance(consequent_items, list):
        return any(re.match(diagnosis_pattern, item) for item in consequent_items)
    return False

# Reglas donde el consecuente es un diagnóstico
df_filtrado = df_spark.filter(has_diagnosis_udf(col("consequent")))


In [6]:
# Contar el número de antecedentes
df_filtrado = df_filtrado.withColumn("num_antecedents", size(col("antecedent")))

# Filtrar reglas con 5 o menos antecedentes
df_final = df_filtrado.filter(col("num_antecedents") <= 5)


In [7]:
df_final.show(10, truncate=False)

+-----------------------------------+----------+------------------+------------------+--------------------+---------------+
|antecedent                         |consequent|confidence        |lift              |support             |num_antecedents|
+-----------------------------------+----------+------------------+------------------+--------------------+---------------+
|[50862, 50983, 50902, 51006, 51279]|[E87]     |0.7212218056665233|3.2029281341216995|0.056164717932028006|5              |
|[50821, 50971, 51237, 51006, 51279]|[E87]     |0.7024684379121915|3.1196448934903795|0.05352758460603334 |5              |
|[I13, I50, 51249, 51279]           |[N18]     |0.9943776489923017|5.055214764417688 |0.055020843404055726|4              |
|[50820, 50983, 52172, 51221, 51222]|[E87]     |0.7108295664353473|3.1567764576907633|0.05343664897410249 |5              |
|[I48, E78, 52172, 51221, 50931]    |[Z79]     |0.730888082762327 |2.2121452801454065|0.05207261449513973 |5              |
|[50821,

In [8]:
# Mostrar la cantidad de reglas después del filtrado
print(f"Después del filtrado hay {df_final.count()} reglas con diagnósico como consecuente")

Después del filtrado hay 8155 reglas con diagnósico como consecuente


In [ ]:
# Guardar en formato Parquet
ruta_salida_parquet = "/data/reglas_diagnostico_filtradas_5antecedent.parquet"

df_final.coalesce(1).write.mode("overwrite").parquet("/data/reglas_diagnostico_filtradas_5antecedent.parquet")

print(f"Archivo guardado en: {ruta_salida_parquet}")